# Getting started with metasyn

In this tutorial, we will be creating a `MetaFrame`, which is a metadata representation of a given dataset, and proceed by generating synthetic data from it. This example workflow starts from a `.csv` file as input, but it easily adapted to other formats.  

## Step 0: Install the metasyn package and import required packages
First, install the metasyn package in your session:

In [ ]:
# uncomment the following line and run the cell to install metasyn
# %pip install metasyn

In [ ]:
# import required packages
import datetime as dt
import polars as pl
from metasyn import MetaFrame, demo_file

## Step 1: Load the data into a data frame

The first step in creating the metadata is reading and converting your dataset to a DataFrame with the correct data types. We use the [Polars](https://pola.rs) dataframe library for this (but you could also use Pandas!)

In [ ]:
# get the path of the demo csv
demo_file_path = demo_file()

# read the data with the correct categorical variables
data_types={
    "Sex": pl.Categorical,
    "Embarked": pl.Categorical
}
df = pl.read_csv(demo_file_path, try_parse_dates=True, dtypes=data_types)

# check out the data
df.head()

Now, let's check the data types of our DataFrame:

In [ ]:
dict(zip(df.columns, df.dtypes))

We see that most variables are now nicely specified as strings, categories, dates and ints where necessary. We can also inspect the data a bit more with `describe()`.

In [ ]:
df.describe()

## Step 2: Generating a MetaFrame

Now that we have properly formatted our DataFrame, we can easily generate a MetaFrame for it. 

> **MetaFrames:**
> A **MetaFrame** is an object which captures the essential aspects of the dataset, including variable names, types, data types, the percentage of missing values, and distribution attributes. MetaFrame objects capture all the information needed to generate a synthetic dataset that aligns with the original dataset, without containing any *entries* of the original dataset.

More information on generating MetaFrames can be found on the metasyn docs, [here](https://metasynth.readthedocs.io/en/latest/usage/generating_metaframes.html).

In [ ]:
# Generate and fit a MetaFrame to the DataFrame 
mf = MetaFrame.fit_dataframe(df)

We can call the `print` function to display the (statistical metadata contained in the) MetaFrame in an easy-to-read format:

In [ ]:
print(mf)

## Step 3: Exporting the MetaFrame

After creating the MetaFrame, Metasyn can serialize and export it into a GMF file using `mf.export()`, passing in the filepath as a parameter.

> **GMF files:**
> GMF files are JSON files that follow the [Generative Metadata Format (GMF)](https://github.com/sodascience/generative_metadata_format), a format designed to contain statistical metadata for (tabular) datasets that has been designed to be easy to read and understand. This allows users to audit, understand, modify and share their data generation model with ease.

More information on exporting and importing MetaFrames can be found on the metasyn docs, [here](https://metasynth.readthedocs.io/en/latest/usage/exporting_metaframes.html).

In [ ]:
file_path = "demonstration_metadata.json"

# Serialize and export the MetaFrame to a GMF file
mf.export(file_path)

You can now open and read the GMF formatted .json file!

A (previously exported) GMF file can be imported and loaded into a MetaFrame using the `MetaFrame.from_json()` class method, passing in the file path as a parameter. 

In [ ]:
# Create a MetaFrame based on a GMF (.json) file
mf = MetaFrame.from_json(file_path)

## Step 4: Generating synthetic data from a MetaFrame

Once a MetaFrame is loaded, synthetic data can be generated from it. The `synthesize` method takes the number of rows to be generated as parameter and returns a DataFrame with the synthetic data.

More information on generating synthetic data based on MetaFrames can be found on the metasyn docs, [here](https://metasynth.readthedocs.io/en/latest/usage/generating_synthetic_data.html).


In [ ]:
# generate synthetic data
mf.synthesize(5)

As you can see, the fake data looks a lot like the real data! However, it could still use some improvement. Below, a brief example is shown of such potential manual improvements. If you want to know more about these improvements, take a look at our [advanced tutorial](https://colab.research.google.com/github/sodascience/metasyn/blob/main/examples/advanced_tutorial.ipynb). 

In [ ]:
from metasyn.distribution import DiscreteUniformDistribution, RegexDistribution, FakerDistribution

# Using some advanced features of metasyn
var_spec = {
    # Ensure that the passengerId column is unique
    "PassengerId": {"unique": True}, 
    # Use fake names for the name column
    "Name": {"distribution": FakerDistribution("name")}, 
     # Estimate / fit an exponential distribution
    "Fare": {"distribution": "LogNormalDistribution"},
    # Manually set a distribution for age 
    "Age": {"distribution": DiscreteUniformDistribution(20, 40)},
    # Manually set a regex distribution for cabin
    "Cabin": {"distribution": RegexDistribution(r"[ABCDEF][0-9]{2,3}")}
}

# create the high-quality metadata
mf = MetaFrame.fit_dataframe(df, spec=var_spec)

# generate synthetic data
syn_df = mf.synthesize(len(df))
syn_df.head()

Now, let's compare the synthetic data to the real data:

In [ ]:
df.describe()

In [ ]:
syn_df.describe()